## Core Functions
- Report the range of times for each event for the given data set (done)
- Report the median time for each for the selected data set (done)
- Report fastest for each event along with that person and the rest of their time (EX "Christina Hepper has the fastest Swim at _____ and had the following other times...., Jamie Fend had the fastest T1 at _____ and had teh following other times...." (done, but need to improve formatting of result)
- Create functions
-- function to convert to integer minutes (done)
-- function for performance standardizing (done)
- Compute the fastest theoretical time (done)
- Compute the median theoretical time
- Report how selected Bib Number did relative to lead (EX "989 was 0.9x at swimming, 1.1x at T1....")
- Create a line plot with each event on the X, the Standardized Time on the Y (standardize to fastest?) and each line being an athlete, highlight the user and the median line
- Need to cumsum the colums

## Future
- Use mongo db to store data
- Automatically scrape the results
- Compute a "competitiveness" score for each race (like FSAE world: https://fs-world.org/)
- Calculate the percentile for eahc event for the user

In [103]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import time
#from bokeh.plotting import figure, show
#from sqlalchemy import create_engine
#pd.set_option("precision", 2)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)

Read in the data from the spreadsheet

In [37]:
data = pd.read_excel('half-moon-bay-olympic.xlsx', header = 0, index_col=None)
data.head()
#
#testdf = pd.read_html("https://results.svetiming.com/Santa-Cruz-Triathlon/events/2021/Santa-Cruz-Triathlon/results", attrs={'class': 'table table-condensed table-bordered table-striped table-hover'})

,Unnamed: 0,Bib,Name,Team Name,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
0,0,327,BEN HOSKINS,STANFORD TRI CLUB,07:00:00,00:14:34,00:01:27,00:59:22,00:00:56,00:31:14,01:47:35,OLYMPIC TRIATHLON,1,23,1,M,1
1,1,312,ALEKSEI AVERCHENKO,TEAM SHEEPER,07:00:00,00:12:59,00:01:03,01:02:03,00:00:43,00:30:55,01:47:46,OLYMPIC TRIATHLON,2,29,1,M,2
2,2,30,KYLE WILLIAMS,THE OLYMPIC CLUB,07:05:00,00:14:14,00:02:09,00:59:03,00:01:01,00:35:43,01:52:12,OLYMPIC TRIATHLON,3,32,1,M,3
3,3,279,ERIC LAURICELLA,NaN,07:00:00,00:17:05,00:01:04,01:00:56,00:01:12,00:32:44,01:53:03,OLYMPIC TRIATHLON,4,26,2,M,4
4,4,101,PATRICK TALBOTT,NaN,07:05:00,00:15:00,00:02:14,01:01:40,00:01:17,00:34:04,01:54:17,OLYMPIC TRIATHLON,5,33,2,M,5


Clean up the columns

In [38]:
data.drop(data.columns[[0]], axis=1, inplace=True)


In [39]:
data.head()
#data['Age'].describe()

,Bib,Name,Team Name,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
0,327,BEN HOSKINS,STANFORD TRI CLUB,07:00:00,00:14:34,00:01:27,00:59:22,00:00:56,00:31:14,01:47:35,OLYMPIC TRIATHLON,1,23,1,M,1
1,312,ALEKSEI AVERCHENKO,TEAM SHEEPER,07:00:00,00:12:59,00:01:03,01:02:03,00:00:43,00:30:55,01:47:46,OLYMPIC TRIATHLON,2,29,1,M,2
2,30,KYLE WILLIAMS,THE OLYMPIC CLUB,07:05:00,00:14:14,00:02:09,00:59:03,00:01:01,00:35:43,01:52:12,OLYMPIC TRIATHLON,3,32,1,M,3
3,279,ERIC LAURICELLA,NaN,07:00:00,00:17:05,00:01:04,01:00:56,00:01:12,00:32:44,01:53:03,OLYMPIC TRIATHLON,4,26,2,M,4
4,101,PATRICK TALBOTT,NaN,07:05:00,00:15:00,00:02:14,01:01:40,00:01:17,00:34:04,01:54:17,OLYMPIC TRIATHLON,5,33,2,M,5


In [40]:
#data_test = pd.to_datetime(data['Start Time'])

In [41]:
#data_test.head()
#print(data.at[1, 'Swim'])

The values all come in as intergs. The code below converts the data to decimal time

It also creates cumulative times for later use

In [42]:
#create a function to convert the time to integer seconds
#To create a new column, use the [] brackets with the new column name at the left side of the assignment.

def convertTime (time):
    temp = time.split(':')
    timeMinutes = (int(temp[0])*60)+int(temp[1])+int(temp[2])/60
    return timeMinutes

    
#convert to integers
data["Swim Minutes"] = data["Swim"].apply(convertTime)
data["T1 Minutes"] = data["T1"].apply(convertTime)
data["Bike Minutes"] = data["Bike"].apply(convertTime)
data["T2 Minutes"] = data["T2"].apply(convertTime)
data["Run Minutes"] = data["Run"].apply(convertTime)
#data["Elapsed Minutes"] = data["Chip Elapsed"].apply(convertTime)

#create cumulative times
data["Swim+T1"]=data["Swim Minutes"]+data["T1 Minutes"]
data["Plus Bike"]=data["Swim+T1"]+data["Bike Minutes"]
data["Plus T2"]=data["Plus Bike"]+data["T2 Minutes"]
data["Total"]=data["Plus T2"]+data["Run Minutes"]



data.head()

,Bib,Name,Team Name,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,...,Gender Place,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes,Swim+T1,Plus Bike,Plus T2,Total
0,327,BEN HOSKINS,STANFORD TRI CLUB,07:00:00,00:14:34,00:01:27,00:59:22,00:00:56,00:31:14,01:47:35,...,1,14.57,1.45,59.37,0.93,31.23,16.02,75.38,76.32,107.55
1,312,ALEKSEI AVERCHENKO,TEAM SHEEPER,07:00:00,00:12:59,00:01:03,01:02:03,00:00:43,00:30:55,01:47:46,...,2,12.98,1.05,62.05,0.72,30.92,14.03,76.08,76.80,107.72
2,30,KYLE WILLIAMS,THE OLYMPIC CLUB,07:05:00,00:14:14,00:02:09,00:59:03,00:01:01,00:35:43,01:52:12,...,3,14.23,2.15,59.05,1.02,35.72,16.38,75.43,76.45,112.17
3,279,ERIC LAURICELLA,NaN,07:00:00,00:17:05,00:01:04,01:00:56,00:01:12,00:32:44,01:53:03,...,4,17.08,1.07,60.93,1.20,32.73,18.15,79.08,80.28,113.02
4,101,PATRICK TALBOTT,NaN,07:05:00,00:15:00,00:02:14,01:01:40,00:01:17,00:34:04,01:54:17,...,5,15.00,2.23,61.67,1.28,34.07,17.23,78.90,80.18,114.25


Count how many people in each ag group

In [44]:
age18_19 = data[(data['Age'] <= 19) & (data['Age'] >= 18)]['Age'].count()
age20_24 = data[(data['Age'] <= 20) & (data['Age'] >= 24)]['Age'].count()

print(age18_19)
print(age20_24)

2


In [71]:
start_ages = []
end_ages =[]
for start in range(20, 85, 5):
    start_ages.append(start)
for end in range (24, 89, 5):
    end_ages.append(end)

print(start_ages)
print(end_ages)

[20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
[24, 29, 34, 39, 44, 49, 54, 59, 64, 69, 74, 79, 84]


In [99]:
def determine_agegroup(row):
    age = int(row['Age'])
    for start, stop in zip(start_ages, end_ages):
        if start<=age<=stop:
            return '%d-%d' %(start, stop)


In [100]:
data['Age Group'] = data.apply(determine_agegroup, axis=1)

In [108]:
type(data.at[0, 'Age'])

numpy.int64

In [82]:
group_sizes = []
#group_sizes.append(data[(data['Age'] <= 19) & (data['Age'] >= 18)]['Age'].count())
for start, stop in zip(start_ages, end_ages):
    group_sizes.append(data[(data['Age'] <= stop) & (data['Age'] >= start)]['Age'].count())
#group_sizes.append(group_sizes.append(data[(data['Age'] >= 85)]['Age'].count()))

In [83]:
print(len(group_sizes))
print(group_sizes)

13
[18, 74, 78, 39, 28, 35, 27, 18, 9, 5, 1, 1, 0]


In [84]:

ages = [ '20-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74', '75-79', '80-84',]
print(len(ages))

13


In [90]:
zipped = zip(ages, group_sizes)
piedf = pd.DataFrame(zipped, columns=['Age Range', 'Athletes'])
piedf

,Age Range,Athletes
0,20-24,18
1,25-29,74
2,30-34,78
3,35-39,39
4,40-44,28
5,45-49,35
6,50-54,27
7,55-59,18
8,60-64,9
9,65-69,5


In [92]:
fig = px.pie(piedf, values='Athletes', names='Age Range')
fig.show()

Filter out the people who didn't finish, have zeros for times and optionally were on teams and by gender

In [19]:
data = data[data['Gender'] == 'F']  # set M or F for gener or comment out line to include all
#data = data[data.isnull().any(axis=1)] #do this to filter in people who were not on a team
data = data[data['Run Minutes'] > 0]
data = data[data['Bike Minutes'] > 0]


Get statistical data about the event.

In [20]:
summary = data.describe()
data[["Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]].describe()


,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
count,29.00,29.00,29.00,29.00,29.00
mean,22.43,5.17,81.79,3.31,50.36
std,3.14,1.90,6.68,1.47,7.35
min,17.73,2.27,70.95,1.42,35.63
25%,20.32,3.78,76.03,2.25,46.02
50%,22.33,4.63,81.50,3.13,49.73
75%,23.58,6.30,87.17,3.88,55.47
max,31.60,10.45,93.92,7.12,66.90


In [21]:
#assign user variable
user = "SAMANTHA DEWEES"
index = data.index[data['Name']==user].tolist()
data.loc[index, ["Name", "Age", "Gender Place", "Swim", "T1", "Bike", "T2", "Run"]]

,Name,Age,Gender Place,Swim,T1,Bike,T2,Run
167,SAMANTHA DEWEES,26,30,00:22:23,00:04:38,01:16:02,00:03:12,00:44:41


In [22]:
data['Swim Minutes'].idxmin()
data

,Bib,Name,Team Name,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,...,Gender Place,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes,Swim+T1,Plus Bike,Plus T2,Total
63,404,MADELINE MARINO,NaN,07:05:00,00:17:44,00:02:16,01:10:57,00:02:00,00:40:41,02:13:41,...,3,17.73,2.27,70.95,2.00,40.68,20.00,90.95,92.95,133.63
67,217,CHRISTINE EDMONDS,NaN,07:05:00,00:19:25,00:03:30,01:11:25,00:01:25,00:38:04,02:13:51,...,4,19.42,3.50,71.42,1.42,38.07,22.92,94.33,95.75,133.82
99,123,REIKO AKIMOTO,NaN,07:05:00,00:19:19,00:03:18,01:11:41,00:01:36,00:43:30,02:19:25,...,11,19.32,3.30,71.68,1.60,43.50,22.62,94.30,95.90,139.40
110,204,KAREN WEIDERT,NaN,07:10:00,00:22:17,00:02:49,01:13:18,00:01:44,00:40:19,02:20:29,...,14,22.28,2.82,73.30,1.73,40.32,25.10,98.40,100.13,140.45
112,122,ANNMARIE MOONEY,NaN,07:00:00,00:19:59,00:03:47,01:18:05,00:03:14,00:35:38,02:20:45,...,15,19.98,3.78,78.08,3.23,35.63,23.77,101.85,105.08,140.72
153,219,KATHRYN MOONEY,NaN,07:05:00,00:22:26,00:03:50,01:14:44,00:03:45,00:44:43,02:29:30,...,25,22.43,3.83,74.73,3.75,44.72,26.27,101.00,104.75,149.47
162,435,VICTORIA NORMAN,NaN,07:05:00,00:21:52,00:02:52,01:15:54,00:01:51,00:47:54,02:30:25,...,27,21.87,2.87,75.90,1.85,47.90,24.73,100.63,102.48,150.38
167,353,SAMANTHA DEWEES,NaN,07:00:00,00:22:23,00:04:38,01:16:02,00:03:12,00:44:41,02:30:58,...,30,22.38,4.63,76.03,3.20,44.68,27.02,103.05,106.25,150.93
179,456,AUGUST FERN,NaN,07:10:00,00:20:48,00:02:52,01:19:12,00:01:59,00:47:07,02:32:00,...,32,20.80,2.87,79.20,1.98,47.12,23.67,102.87,104.85,151.97
190,441,SHELBY DOLLIVER,NaN,07:05:00,00:18:07,00:04:32,01:22:52,00:02:35,00:46:01,02:34:08,...,34,18.12,4.53,82.87,2.58,46.02,22.65,105.52,108.10,154.12


In [23]:
def getMinTime (stage):
    minTime = data[stage].min()
    minIndex = data[stage].idxmin()
    #print("minTime is", minTime)
    #print("minIndex is", minIndex)
    return minTime, minIndex

def printResults (minIndex):
    #need to make this so it doesnt print reduntant colums or prints something nicely formatted
    print(data.loc[minIndex,:])

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 63 to 307
Data columns (total 25 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Bib           29 non-null     int64  
 1   Name          29 non-null     object 
 2   Team Name     0 non-null      object 
 3   Start Time    29 non-null     object 
 4   Swim          29 non-null     object 
 5   T1            29 non-null     object 
 6   Bike          29 non-null     object 
 7   T2            29 non-null     object 
 8   Run           29 non-null     object 
 9   Chip Elapsed  29 non-null     object 
 10  Division      29 non-null     object 
 11  Div Place     29 non-null     int64  
 12  Age           29 non-null     int64  
 13  Age Place     29 non-null     int64  
 14  Gender        29 non-null     object 
 15  Gender Place  29 non-null     int64  
 16  Swim Minutes  29 non-null     float64
 17  T1 Minutes    29 non-null     float64
 18  Bike Minutes  29 non-null     

In [25]:
events = ['Swim Minutes', 'T1 Minutes', 'Bike Minutes', 'T2 Minutes', 'Run Minutes']

shortestTime = 0
for stage in events:
    minTime, minIndex = getMinTime(stage)
    print("the fastest for ", stage)
    #print(minIndex)
    # print(data.loc[[minIndex]])
    printResults(minIndex)
    # shortestTime = shortestTime + minTime

print("the shortest possible time is: ", shortestTime)
#need to work on formatting

the fastest for  Swim Minutes
Bib                           404
Name              MADELINE MARINO
Team Name                     NaN
Start Time               07:05:00
Swim                     00:17:44
T1                       00:02:16
Bike                     01:10:57
T2                       00:02:00
Run                      00:40:41
Chip Elapsed             02:13:41
Division        OLYMPIC TRIATHLON
Div Place                      64
Age                            31
Age Place                       1
Gender                          F
Gender Place                    3
Swim Minutes                17.73
T1 Minutes                   2.27
Bike Minutes                70.95
T2 Minutes                   2.00
Run Minutes                 40.68
Swim+T1                     20.00
Plus Bike                   90.95
Plus T2                     92.95
Total                      133.63
Name: 63, dtype: object
the fastest for  T1 Minutes
Bib                           404
Name              MADELINE MARINO


In [26]:
reduced2 = data[["Name","Swim Minutes","Swim+T1","Plus Bike","Plus T2","Total","Gender Place"]]
reduced2["Start"] = 0
# reduced2 = reduced2.drop(25)
# reduced2

# color_nums = reduced2['Gender Place'].tolist()
# color_names = reduced2['Name'].tolist()
# colors_zip = zip(color_nums, color_names)

# colors_zip

reduced2.head()

/var/folders/83/24d95jlj0_194_rv0vcx1j7r0000gp/T/ipykernel_28677/895480473.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Name,Swim Minutes,Swim+T1,Plus Bike,Plus T2,Total,Gender Place,Start
63,MADELINE MARINO,17.73,20.00,90.95,92.95,133.63,3,0
67,CHRISTINE EDMONDS,19.42,22.92,94.33,95.75,133.82,4,0
99,REIKO AKIMOTO,19.32,22.62,94.30,95.90,139.40,11,0
110,KAREN WEIDERT,22.28,25.10,98.40,100.13,140.45,14,0
112,ANNMARIE MOONEY,19.98,23.77,101.85,105.08,140.72,15,0


In [27]:
dimensions = list([
            dict(range = [0, 1],
                label = 'Start', values = reduced2['Start']),            
            dict(range = [reduced2["Swim Minutes"].min(), reduced2["Swim Minutes"].max()],
                label = 'Time After Swim', values = reduced2['Swim Minutes']),
            dict(range = [reduced2["Swim+T1"].min(), reduced2["Swim+T1"].max()],
                label = 'Time After First Transition', values = reduced2['Swim+T1']),
            dict(range = [reduced2["Plus Bike"].min(), reduced2["Plus Bike"].max()],
                label = 'Time After Bike', values = reduced2['Plus Bike']),
            dict(range = [reduced2["Plus T2"].min(), reduced2["Plus T2"].max()],
                label = 'Time After Second Transition', values = reduced2['Plus T2']),
            dict(range = [reduced2["Total"].min(), reduced2["Total"].max()],
                label = 'Total Time', values = reduced2['Total']),
            dict(range=[0,reduced2['Gender Place'].max()], tickvals = reduced2['Gender Place'], ticktext = reduced2['Name'],
                       label='Competitor', values=reduced2['Gender Place'])
        ])

fig = go.Figure(data=go.Parcoords(line = dict(color = reduced2['Gender Place'],
                   colorscale = [[.0,'rgba(255,0,0,0.1)'],[0.2,'rgba(0,255,0,0.1)'],[.4,'rgba(0,0,255,0.1)'], 
                                 [.6,'rgba(0,255,255,0.1)'], [.8, 'rgba(255,0,255,0.1)'], [1, 'rgba(0,0,0,0.1)']]), dimensions=dimensions))

fig.update_layout(
    title="Triathalon Results",
    width=1920,
    height=1080
)

fig.show()

In [ ]:
https://stackoverflow.com/questions/64139316/plotly-how-to-insert-a-categorical-variable-into-a-parallel-coordinates-plot

                   colorscale = [[.0,'rgba(218,79,79,0.1)'],[0.25,'rgba(148,218,79,0.1)'],[.5,'rgba(79,218,218,0.1)'], 
                                 [.75,'rgba(148,79,218,0.1)'], [1, 'rgba(0,0,0,0.1)']]), dimensions=dimensions))

In [ ]:
engine = create_engine('sqlite:///data.db', echo=False)

In [ ]:
sql_df = pd.read_sql('data.db', con=engine, index_col='ID')

In [ ]:
sql_df.head()

In [ ]:
scatter = px.scatter(x=data['Age'], y=data['Gender Place'])
scatter.show()
